# SISTEMA DE RECOMENDACION USANDO AGRUPACION Y KNN

## TRATAMIENTO DE DATOS

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.cluster import DBSCAN

In [5]:
df = pd.read_csv('musica.csv')
df.head()

,Nombre,Pop,Indie,música clásica,rock ingles,Rock español,Reggaeton,Metal,Corridos,Reggae,...,Tecno,Folk,Disco,Himnos,Salsa,Merengue,kpop,Bachata,Vallenato,Rap
0,Cristian guevara,4,2,7,5,4,10,0,6,0,...,1,1,3,1,10,9,1,8,7,1
1,Yajhaira Naranjo,9,5,9,7,4,10,2,9,7,...,2,5,3,2,10,10,0,10,10,3
2,Sebastian Mosquera,7,9,10,9,9,1,1,3,7,...,3,7,8,7,3,7,1,4,5,4
3,Juan Andrés López,7,9,5,10,10,10,10,2,1,...,6,7,9,1,10,10,5,9,6,6
4,Andrea Torrente,10,7,4,3,10,10,1,4,8,...,2,2,7,10,7,8,3,10,8,9


In [6]:
df.describe()

,Pop,Indie,música clásica,rock ingles,Rock español,Reggaeton,Metal,Corridos,Reggae,Banda,...,Tecno,Folk,Disco,Himnos,Salsa,Merengue,kpop,Bachata,Vallenato,Rap
count,20.000000,20.000000,20.000000,20.000000,20.00000,20.00000,20.000000,20.000000,20.000000,20.000000,...,20.000000,20.000000,20.0000,20.000000,20.000000,20.000000,20.000000,20.000000,20.00000,20.000000
mean,7.500000,6.500000,5.900000,7.650000,7.45000,7.60000,3.600000,4.600000,5.050000,4.900000,...,4.950000,4.750000,6.3000,2.650000,7.600000,6.750000,3.050000,7.050000,5.20000,7.050000
std,1.701393,3.120391,2.425739,1.980829,2.32775,3.34664,3.050453,3.283131,2.564433,2.918183,...,2.762055,2.244877,2.7549,3.013566,2.872739,2.788605,3.017057,3.017057,3.13889,2.684752
min,4.000000,2.000000,3.000000,3.000000,3.00000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.0000,0.000000,1.000000,1.000000,0.000000,0.000000,0.00000,1.000000
25%,7.000000,3.750000,4.000000,7.000000,5.00000,6.00000,1.000000,1.750000,4.000000,2.750000,...,2.000000,2.000000,4.0000,1.000000,5.000000,4.750000,1.000000,5.000000,3.50000,5.500000
50%,7.000000,6.500000,5.000000,8.000000,8.00000,10.00000,3.500000,4.500000,5.500000,5.000000,...,6.000000,5.000000,7.0000,2.000000,9.000000,8.000000,2.500000,8.000000,5.50000,8.000000
75%,9.000000,9.250000,7.250000,9.250000,9.25000,10.00000,5.000000,7.000000,7.000000,7.250000,...,7.000000,7.000000,8.0000,3.000000,10.000000,8.250000,3.500000,10.000000,8.00000,9.000000
max,10.000000,10.000000,10.000000,10.000000,10.00000,10.00000,10.000000,10.000000,8.000000,9.000000,...,10.000000,8.000000,10.0000,10.000000,10.000000,10.000000,10.000000,10.000000,10.00000,10.000000


In [7]:
df.columns

Index(['Nombre', 'Pop', 'Indie', 'música clásica', 'rock ingles',
       'Rock español', 'Reggaeton', 'Metal', 'Corridos', 'Reggae', 'Banda',
       'Jazz', 'Ska', 'Popular', 'Electro', 'Trance', 'Tecno', 'Folk', 'Disco',
       'Himnos', 'Salsa', 'Merengue ', 'kpop', 'Bachata', 'Vallenato', 'Rap'],
      dtype='object')

In [8]:
# dataframe con las recomendaciones de artistas
artistas = pd.read_csv('artistas.csv')
artistas.head()

,Nombre,Pop,Indie,música clásica,rock ingles,Rock español,Reggaeton,Metal,Corridos,Reggae,...,Tecno,Folk,Disco,Himnos,Salsa,Merengue,kpop,Bachata,Vallenato,Rap
0,Big Chungus,0,1,2,3,4,5,5,0,6,...,9,5,4,3,8,9,8,7,6,5
1,Ronaldo Eferrero,7,6,7,5,4,9,6,5,4,...,5,4,3,6,7,8,8,6,5,4
2,Chaco Pedron,9,8,6,9,0,9,8,9,6,...,6,1,7,8,9,8,6,9,1,8
3,Pancracio Puto,9,8,2,3,5,4,3,5,4,...,7,5,8,4,3,2,8,9,0,2
4,Pen Ismall,8,7,6,4,8,9,0,2,8,...,7,6,4,3,6,8,9,7,5,3


In [9]:
# retorna dataframes sin modificar

def get_main_df():
    return df

def get_recommend_df():
    return artistas


In [10]:
# Usuario selecionado a comparar con dataframe original
user_select = 'Sebastian Mosquera'

# Usuario agregado, resultado del anterior, a comparar con dataframe de sugerencias
user_agr = 0

In [11]:
# Retorna o modifica usuario seleccionado

def get_user_select():
    return user_select

def set_user_select(x):
    user_select = x
    return 1

In [12]:
# como crear un dict
pesos = {
    'Pop': 0.1,
    'Indie': 0.1
}
# como agregar vals
pesos['Pee'] = 0.1



# crear diccionario con los pesos de todas las columnas (excepto Nombre)
pesos = {}

for i in df.drop(columns='Nombre').columns:
        pesos[i] = 1
        
print(pesos)

{'Pop': 1, 'Indie': 1, 'música clásica': 1, 'rock ingles': 1, 'Rock español': 1, 'Reggaeton': 1, 'Metal': 1, 'Corridos': 1, 'Reggae': 1, 'Banda': 1, 'Jazz': 1, 'Ska': 1, 'Popular': 1, 'Electro': 1, 'Trance': 1, 'Tecno': 1, 'Folk': 1, 'Disco': 1, 'Himnos': 1, 'Salsa': 1, 'Merengue ': 1, 'kpop': 1, 'Bachata': 1, 'Vallenato': 1, 'Rap': 1}


In [13]:
# Retornar o cambiar dict de pesos
def get_pesos():
    return pesos

def set_pesos(x):
    pesos = x
    return 1

In [14]:
# Asegurar valores como float

for i in df.drop(columns='Nombre').columns:
    df[i] = pd.to_numeric(df[i], downcast='float')

for i in artistas.drop(columns='Nombre').columns:
    artistas[i] = pd.to_numeric(artistas[i], downcast='float')

In [15]:
print(type(df.Pop[0]))

<class 'numpy.float32'>


In [16]:
# copia dataframes

dfCopy = df.copy()
artistasCopy = artistas.copy()

In [17]:
## normalizar valores

# solucion satanica
dfCopy = dfCopy.drop(columns="Nombre")
dfCopy = (dfCopy-dfCopy.min())/(dfCopy.max()-dfCopy.min())
dfCopy.insert(0, "Nombre", df['Nombre'])

artistasCopy = artistasCopy.drop(columns="Nombre")
artistasCopy = (artistasCopy-artistasCopy.min())/(artistasCopy.max()-artistasCopy.min())
artistasCopy.insert(0, "Nombre", artistas['Nombre'])

## CORRELACION ENTRE USUARIOS

#### Aplicacion de pesos

In [18]:
# multiplica las columnas con su llave respectiva en el diccionario de pesos (Pop con peso de Pop...)
for i in dfCopy.drop(columns="Nombre").columns:
    dfCopy[i] = dfCopy[i]*pesos[i]

In [19]:
# retorna dataframe normalizado
def get_norm_df():
    return dfCopy

#### Correlacion

In [20]:
import sklearn.metrics.pairwise 

In [21]:
# Como encontrar fila con valor de columna (cuando el nombre sea igual al usuario)
dfCopy.loc[dfCopy['Nombre']==user_select]

# Lo mismo pero con valor solo conocido por el indice
dfCopy.loc[dfCopy['Nombre']== dfCopy.iloc[1].Nombre]

# Funcion de similitud coseno
cos_sim = sklearn.metrics.pairwise.cosine_similarity

# Iterador del tamaño de la dimension vertical correlacionando cada fila con usuario seleccionado y agregandolo al diccionario
corrs = {}

for i in range(dfCopy.shape[0]):
    corrs[dfCopy.iloc[i].Nombre] = cos_sim(dfCopy.loc[dfCopy['Nombre']==user_select].drop(columns='Nombre') , 
                                                                          dfCopy.loc[dfCopy['Nombre']==dfCopy.iloc[i].Nombre].drop(columns='Nombre'))[0][0] 
print(corrs)

{'Cristian guevara': 0.5691713, 'Yajhaira Naranjo': 0.77768576, 'Sebastian Mosquera': 1.0000001, 'Juan Andrés López': 0.7865738, 'Andrea Torrente': 0.71551126, 'Tatiana Campos': 0.6754133, 'Isabella Pinzon': 0.76916623, 'Christian Aparicio': 0.6786546, 'Luisa Tapiero': 0.75759244, 'Nicolás Penagos': 0.80762136, 'Alejandro Medina': 0.71237457, 'Valeria Jiménez ': 0.86470515, 'Juan Rodriguez': 0.6652869, 'Juan Agredo ': 0.8438932, 'Daniel Martinez': 0.697616, 'Christian Castro': 0.7827977, 'Juan Pablo Bueno': 0.7687861, 'Juan Camilo Munera': 0.75377244, 'Nicolas Orozco': 0.79740924, 'Jose Moncada': 0.84282905}


In [22]:
# retorna diccionario de correlaciones con usuario seleccionado
def get_corrs():
    return corrs

#### Vecindarios

In [23]:
# Numero de usuarios a considerar como vecinos cercanos
num_vec = 5

# Metodo de agrupacion elegido: 0 = Naive Average, 1 = Least Misery, 2 = Maximum Pleasure, 3 = Media Satisfaction

agr_met = 0

In [24]:
# retorna y signa num usuarios y metodo

def get_num_vec():
    return num_vec

def get_agr_met():
    return agr_met

def set_num_vec(x):
    if(x>0 and x<dfCopy.shape[0]):
        num_vec = x
        return 1
    else:
        return 0

def set_agr_met(x):
    if(x>0 and x<4):
        agr_met = x
        return 1
    else:
        return 0

In [25]:
# Obtener top N valores de un diccionario D
def top_val(N, D):
    temp = D.copy()
    final = {}
    
    # animal satanico para obtener llave con un valor, en este caso, el valor maximo
    # print(list(temp.keys())[list(temp.values()).index(max(temp.values()))])
    
    # iterador que agrega los N valores mas grandes del diccionario al diccionario final
    
    for i in range(N+1):
        new = list(temp.keys())[list(temp.values()).index(max(temp.values()))]
        final[new] = temp[new]
        temp.pop(new)
    return final

In [26]:
vecinos = top_val(num_vec, corrs)

# borra el usuario seleccionado
vecinos.pop(user_select)

print(vecinos)

{'Valeria Jiménez ': 0.86470515, 'Juan Agredo ': 0.8438932, 'Jose Moncada': 0.84282905, 'Nicolás Penagos': 0.80762136, 'Nicolas Orozco': 0.79740924}


In [27]:
# Retorna vecinos
def get_vecinos():
    return vecinos

## AGREGACION

#### crear protopersona

In [28]:
# dataframe vacio
user_agr = pd.DataFrame(data=None, columns=dfCopy.drop(columns='Nombre').columns)
# primer fila en 0
user_agr.loc[len(user_agr)] = 0

In [29]:
# obtiene los dataframes de las personas en el diccionario vecinos y los une en la protopersiona
for i in vecinos.keys():
    dfI = dfCopy.loc[dfCopy['Nombre']==i].drop(columns='Nombre')
    user_agr = user_agr.add(dfI, fill_value=0)

print(user_agr)

         Pop  Indie  música clásica  rock ingles  Rock español  Reggaeton  \
0   0.000000  0.000        0.000000     0.000000      0.000000        0.0   
9   0.666667  0.125        0.285714     0.714286      0.714286        1.0   
11  0.166667  1.000        0.714286     0.714286      1.000000        0.3   
13  0.833333  0.875        0.142857     1.000000      1.000000        0.3   
18  0.500000  1.000        0.000000     0.571429      1.000000        0.6   
19  0.333333  0.500        0.571429     0.714286      0.714286        1.0   

    Metal  Corridos  Reggae     Banda  ...  Tecno      Folk  Disco  Himnos  \
0     0.0       0.0   0.000  0.000000  ...    0.0  0.000000    0.0     0.0   
9     0.3       1.0   1.000  0.555556  ...    0.7  0.857143    0.8     1.0   
11    0.1       0.1   1.000  0.222222  ...    0.6  1.000000    0.2     0.2   
13    0.8       0.7   0.625  0.666667  ...    0.4  0.857143    0.8     0.2   
18    0.6       0.5   0.625  0.555556  ...    0.6  0.142857    1.0    

In [30]:
# retornar dataframe de vecindario
def get_vecindario_df():
    return user_agr

#### Aplica metodo

In [31]:
# 0 = Naive Average, 1 = Least Misery, 2 = Maximum Pleasure, 3 = Media Satisfaction
agr_met = 3

if (agr_met == 0):
    # se deja el promedio
    user_agr = user_agr.mean()
    
elif (agr_met == 1):
    # se borran columnas menores de 0.5
    user_agr = user_agr.mean()
    for i in user_agr.keys():
        if(user_agr[i] < 0.5):
            user_agr[i] = 0
            
elif (agr_met == 2):
    # se borran columnas menores de 0.8
    user_agr = user_agr.mean()
    for i in user_agr.keys():
        if(user_agr[i] < 0.8):
            user_agr[i] = 0
    
elif (agr_met == 3):
    # se dejan las desviaciones estandar bajas
    
    # arreglo de columnas a "borrar"
    remove = []
    
    for col in user_agr.columns:
        # si la columna no tiene la minima desviacion, agregar a arreglo
        if(user_agr[col].std().min() != user_agr.std().min()):
            remove.append(col)
            
    # borrar las columnas en arreglo
    user_agr[remove[:]] = 0
    
    # ahora si promediamos
    user_agr = user_agr.mean()

else:
    print('wrong number: 0 = Naive Average, 1 = Least Misery, 2 = Maximum Pleasure, 3 = Media Satisfaction')
    
# convertimos usuario a dataframe
user_agr = user_agr.to_frame()

# lo rotamos para que este en el formato correcto
user_agr = user_agr.transpose()

print(user_agr)

   Pop  Indie  música clásica  rock ingles  Rock español  Reggaeton  Metal  \
0  0.0    0.0             0.0          0.0           0.0        0.0    0.0   

   Corridos  Reggae  Banda  ...  Tecno  Folk  Disco  Himnos  Salsa  Merengue   \
0       0.0     0.0    0.0  ...    0.0   0.0    0.0     0.0    0.0        0.0   

   kpop  Bachata  Vallenato  Rap  
0  0.25      0.0        0.0  0.0  

[1 rows x 25 columns]


In [32]:
user_agr.head()

,Pop,Indie,música clásica,rock ingles,Rock español,Reggaeton,Metal,Corridos,Reggae,Banda,...,Tecno,Folk,Disco,Himnos,Salsa,Merengue,kpop,Bachata,Vallenato,Rap
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0


In [33]:
# retornar protopersona final
def get_protopersona():
    return user_agr

## RECOMENDACION

Ahora creamos un vecindario de la protopersona con los perfiles a recomendar, y a la final le recomendamos los mas parecidos... es repetir el paso de correlacion y vecindarios pero con el otro dataframe y en ves de la persona elegida, la protopersona

#### Correlacion

In [34]:
# Numero de recomendaciones
num_rec = 2

In [35]:
# retorna y asigna num recomendaciones

def get_num_rec():
    return num_rec

def set_num_rec(x):
    if(x>0 and x<artistasCopy.shape[0]):
        num_vec = x
        return 1
    else:
        return 0

In [36]:
rec_corrs = {}


for i in range(artistasCopy.shape[0]):
    rec_corrs[artistasCopy.iloc[i].Nombre] = cos_sim(user_agr , artistasCopy.loc[artistasCopy['Nombre']==artistasCopy.iloc[i].Nombre].drop(columns='Nombre'))[0][0] 
print(rec_corrs)

{'Big Chungus': 0.2682847694066756, 'Ronaldo Eferrero': 0.2461392497004484, 'Chaco Pedron': 0.1069717993923396, 'Pancracio Puto': 0.3022152352790845, 'Pen Ismall': 0.29197748063770895, 'Rama Dan': 0.0}


In [37]:
# retorna nivel de correlacion con dataframe de correlacion
def get_corr_protopersona():
    return rec_corrs

#### Vecindarios

In [38]:
# aqui no tenemos usuario repetido...
num_rec = num_rec -1

recomendaciones = top_val(num_rec, rec_corrs)

print(recomendaciones)

{'Pancracio Puto': 0.3022152352790845, 'Pen Ismall': 0.29197748063770895}


In [39]:
# !HHHHHechioho!
# retorna las recomendaciones finales
def get_recomendaciones():
    return recomendaciones